In [29]:
import pandas as pd 
import numpy as np 

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer, MissingIndicator

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor

In [30]:
df = pd.read_csv("../../HackathonNICD/Data/1. Mandata Vehicle Summary.csv")
df = df.dropna()
df = df.reset_index(drop = True)

df_annual = pd.read_csv("../../HackathonNICD/Data/1a. Mandata Vehicle Annual Summary.csv")

In [31]:
def duration_minutes(df): 
    df= df.str.split(":").apply(lambda x: int(x[0]) * 60 + int(x[1]))
    
    return df

In [32]:
col_min = ["DRIVING_TIME", "IDLE_TIME", "STANDING_TIME", "INACTIVE_TIME", "UTILISATION"]
df[col_min] = df[col_min].apply(duration_minutes)

df["mileage"] = df["KM"] / df["LITRES"]
df.loc[~np.isfinite(df['mileage']), 'mileage'] = np.nan
df[df.isnull().any(axis=1)]
df["mileage"].fillna(df["mileage"].mean(), inplace=True)

In [33]:
df_annual[["UTILISATION_TIME"]] = df_annual[["UTILISATION_TIME"]].apply(duration_minutes)

In [34]:
df_annual["util_min"] = df_annual["CO2_KG"] / df_annual["UTILISATION_TIME"] 

In [35]:
df.drop(["ODOMETER_START", "DATE", "ODOMETER_END", "JOURNEYS", "GALLONS", "MPG", "MILES", 
"L_100KM", "STANDING_PERC", "INACTIVE_PERC", "UTILISATION_PERC", "IDLE_PERC", "DRIVE_PERC"], 
axis = 1, inplace = True, errors="ignore")

In [36]:
df["START"] = pd.to_datetime(df["START"])
df["END"] = pd.to_datetime(df["END"])

In [37]:
dfannual_clip = df_annual[["REG","util_min"]]
df_final = pd.merge(df, dfannual_clip, on="REG", how="outer")

In [38]:
df_final["CO2"] = df_final["util_min"] * df_final["UTILISATION"]

In [39]:
df_final.drop(["INACTIVE_TIME", "util_min"], axis = 1, inplace=True, errors="ignore")

In [40]:
def date_time_extract(df, col):
    df[col + "_year"] = df[col].dt.year
    df[col + "_month"] = df[col].dt.month
    df[col + "_day"] = df[col].dt.day
    df[col + "_weekday"] = df[col].dt.weekday
    #df[col + "_time"] = df[col].dt.time 
    df[col + '_hour'] = df[col].dt.hour
    #df[col + '_minute'] = df[col].dt.minute
    #df[col + '_second'] = df[col].dt.second

    return df

In [41]:
date_time_extract(df_final, "START")
date_time_extract(df_final, "END")

,REG,START,END,DRIVING_TIME,IDLE_TIME,STANDING_TIME,UTILISATION,KM,LITRES,mileage,...,START_year,START_month,START_day,START_weekday,START_hour,END_year,END_month,END_day,END_weekday,END_hour
0,DX66OYU,2021-05-27 01:00:00,2021-05-27 18:09:00,386,1,641,1029,264.19,86.13,3.067340,...,2021,5,27,3,1,2021,5,27,3,18
1,DX66OYU,2021-05-28 06:54:00,2021-05-28 20:19:00,518,0,285,804,509.23,170.24,2.991248,...,2021,5,28,4,6,2021,5,28,4,20
2,DX66OYU,2021-06-01 06:54:00,2021-06-02 00:46:00,509,0,561,1071,454.61,142.72,3.185328,...,2021,6,1,1,6,2021,6,2,2,0
3,DX66OYU,2021-06-02 06:25:00,2021-06-02 18:24:00,274,0,444,718,135.77,59.07,2.298459,...,2021,6,2,2,6,2021,6,2,2,18
4,DX66OYU,2021-06-03 07:56:00,2021-06-04 01:00:00,479,3,541,1023,440.18,146.61,3.002387,...,2021,6,3,3,7,2021,6,4,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4491,YM17NCA,2021-12-22 07:00:00,2021-12-23 00:00:00,239,12,767,1019,126.59,71.15,1.779199,...,2021,12,22,2,7,2021,12,23,3,0
4492,YM17NCA,2021-12-23 00:00:00,2021-12-23 15:05:00,291,0,614,905,163.91,63.47,2.582480,...,2021,12,23,3,0,2021,12,23,3,15
4493,YM17NCA,2021-12-24 11:22:00,2021-12-24 11:24:00,1,0,1,2,0.09,0.15,0.600000,...,2021,12,24,4,11,2021,12,24,4,11
4494,YM17NCA,2021-12-30 13:10:00,2021-12-30 14:20:00,0,36,34,70,0.00,1.92,0.000000,...,2021,12,30,3,13,2021,12,30,3,14


In [42]:
df_final.drop(["START_year", "END_year", "START", "END", "DATE"], axis = 1, inplace = True, errors = "ignore")

In [43]:
dummy_reg = pd.get_dummies(df_final["REG"], prefix="REG")
df_final = pd.concat([df_final, dummy_reg], axis=1)

In [44]:
df_final.drop(["REG", "UTILISATION"], axis = 1, inplace = True, errors = "ignore")

In [45]:
x = df_final.drop('CO2', axis = 1)
y = df_final['CO2']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, 
                                                    random_state=42)

In [46]:
col_to_scale = ["DRIVING_TIME", "IDLE_TIME", "STANDING_TIME", "KM", "LITRES", "mileage"]

scaler = MinMaxScaler()
x_train[col_to_scale] = scaler.fit_transform(x_train[col_to_scale])

c:\Users\saqui\anaconda3\envs\ml\lib\site-packages\pandas\core\frame.py:3673: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [47]:
x_test[col_to_scale] = scaler.transform(x_test[col_to_scale])

c:\Users\saqui\anaconda3\envs\ml\lib\site-packages\pandas\core\frame.py:3673: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [48]:
from xgboost import XGBRegressor
reg_xgb = XGBRegressor(colsample_bytree=0.9, learning_rate=0.5)
reg_xgb.fit(x_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.9, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.5, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [49]:
y_pred = reg_xgb.predict(x_test)

In [50]:
print(f"The R2 score is: {r2_score(y_test, y_pred)}")
print(f"The Mean Average Error is: {mean_absolute_error(y_test, y_pred)}")
print(f"The Mean Squared Error is: {mean_squared_error(y_test, y_pred)}")
print(f"The Root Mean Squared Error is: {np.sqrt(mean_squared_error(y_test, y_pred))}")

The R2 score is: 0.9912348879578783
The Mean Average Error is: 7.73478030504605
The Mean Squared Error is: 118.58085266375825
The Root Mean Squared Error is: 10.889483581132682
